In [219]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import random
import datetime
%load_ext tensorboard
from google.cloud import datastore
import os
import numpy as np
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = \
    "/home/alice/Downloads/HCL-customer-application-c05a56fa9ccc.json"

client = datastore.Client()


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Load title and Subject data for prediction

In [220]:
query = client.query(kind="tododata")
x = []
y = []
for i in query.fetch():
    x.append([i['subject'], i['title']])
    y.append([i['totalcoins']])
y = np.asarray(y)

convert both title and subject to one hot encoded matrices because the datasets are both categorical and orthogonal to each other

In [221]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0, lowercase=False)

In [222]:
def toCategorical(data):
    vectorizer.fit(data)
    return vectorizer.transform(data).toarray()

In [223]:
title = toCategorical([i[1] for i in x])

In [224]:
subjects = toCategorical([i[0] for i in x])

In [225]:
x = np.concatenate((titles,subjects),axis=1)

## normalize the prediction values to aid the neural net in fitting the data

In [226]:
def Normalize(data, mean_data =None, std_data =None):
    if not mean_data:
        mean_data = np.mean(data)
    if not std_data:
        std_data = np.std(data)
    norm_data = (data-mean_data)/std_data
    return norm_data, mean_data, std_data

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


Y_train, ymean_data, ystd_data = Normalize(y_train)
Y_test,_,_ = Normalize(y_test, ymean_data, ystd_data)

In [229]:
print(X_train)
print(Y_train)
print(X_test)
print(Y_test)

(6, 13)
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.73205081]
 [-1.73205081]]
[[0 0 1 0 0 0 0 1 0 0 1 1 0]
 [1 1 0 0 1 0 0 0 0 0 0 1 0]]
[[0.        ]
 [1.73205081]]


In [230]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(12, activation='relu',input_shape=(x.shape[1],)))
# Add an output layer with 10 output units:
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

In [231]:
log_dir = "rewardlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X_train, Y_train, 
          validation_data=(X_test,Y_test), 
          batch_size=20, 
          epochs=30,
          verbose=1,
         callbacks=[tensorboard_callback])

Epoch 1/30
1/1 [==============================] - 0s 92ms/step - loss: 1.3695 - mae: 1.0205 - val_loss: 1.4559 - val_mae: 1.0351
Epoch 2/30
1/1 [==============================] - 0s 34ms/step - loss: 1.1741 - mae: 0.8633 - val_loss: 1.2856 - val_mae: 0.9054
Epoch 3/30
1/1 [==============================] - 0s 34ms/step - loss: 1.0481 - mae: 0.7201 - val_loss: 1.1711 - val_mae: 0.7931
Epoch 4/30
1/1 [==============================] - 0s 36ms/step - loss: 0.9737 - mae: 0.5943 - val_loss: 1.0916 - val_mae: 0.7620
Epoch 5/30
1/1 [==============================] - 0s 32ms/step - loss: 0.9248 - mae: 0.5715 - val_loss: 1.0538 - val_mae: 0.7861
Epoch 6/30
1/1 [==============================] - 0s 41ms/step - loss: 0.8830 - mae: 0.6070 - val_loss: 1.0321 - val_mae: 0.8038
Epoch 7/30
1/1 [==============================] - 0s 44ms/step - loss: 0.8412 - mae: 0.6271 - val_loss: 1.0181 - val_mae: 0.8146
Epoch 8/30
1/1 [==============================] - 0s 41ms/step - loss: 0.7847 - mae: 0.6306 - val

In [232]:
%tensorboard --logdir rewardlogs/fit


Reusing TensorBoard on port 6007 (pid 40440), started 0:53:00 ago. (Use '!kill 40440' to kill it.)

LabelEncoder()

array([[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]])